In [5]:
from seleniumbase import Driver
from seleniumbase import page_actions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import os
import requests
import time
import csv

In [6]:
class Skill:
    def __init__(self,name,dir, keyword , groups=None,prerequisites= None):
        filename = name
        filename = filename.replace('/','-')
        filename = filename.replace("\\",'-')
        filename = filename + ".html"
        path = os.path.join(dir, filename)
        path = path.replace("\\",'/')
        self.resource_path = path # for the resource path
        self.keyword_search = keyword  # keyword for searching LLM
        self.group_set = set()
        if groups is not None:    
            self.UpdateGroupSet(groups)
        self.prerequisite  = set()
        if prerequisites is not None:
            self.UpdatePrerequisites(prerequisites)
        

    def UpdateGroupSet(self,groups):
        self.group_set.update(groups)
        #print("skill group set updated.")
      
     
    def UpdatePrerequisites(self,prerequisites):
        self.prerequisite.update(prerequisites)
        #print("prerequisite updated.")
        
class Group:
    def __init__(self,name,skills):
        filename = name
        filename = filename.replace('/','-')
        filename = filename.replace("\\",'-')
        filename = filename + ".html"
        path = os.path.join("group", filename)
        path = path.replace("\\",'/')
        self.resource_path = path # for the resource path
        self.keyword_search = name + " technology" # keyword for searching LLM
        self.skill_set = skills

    def UpdateSkillSet(self,skill):
        self.skill_set.update(skill)
        #print("group skill set updated.")

    def ChangeKeyword(self,keyword):
        self.keyword_search = keyword
    
        

In [7]:
class TechStack:
    def __init__(self):
        self.skill_dict_list = {}
        self.group_dict_list = {}
        self.alias_dict_list = {}
        self.ignore_set = set()
        self.not_found_set = set()

               
    def AddSkillDictList(self,name,keyword,path,groups=None,prerequisites = None):
        if name in self.alias_dict_list:
            name = self.alias_dict_list[name]
            
        if name not in self.skill_dict_list:
            self.skill_dict_list[name] = Skill(keyword,name,path,groups,prerequisites)
            #print(name,"added in skill_dict_list.")
            if groups is not None:
                for g in groups:
                    if g in self.group_dict_list:
                        self.group_dict_list.get(g).UpdateSkillSet({name})
                        #print(name,"added in",g,".")
                    else:
                        self.group_dict_list[g] = Group(g,{name})
                        #print("new group:",g,"have been created and added",name,".")
        else:
            self.UpdateSkillDictList(name,groups,prerequisites)
                    
    def UpdateSkillDictList(self,name,groups,prerequisites):
        if name in self.skill_dict_list:
            self.skill_dict_list[name].UpdateGroupSet(groups)
            if prerequisites is not None:
                self.skill_dict_list[name].UpdatePrerequisites(prerequisites)
                #print(name,"prerequisites updated.")

    def AddGroupDictList(self,name,skills):
        if name in self.group_dict_list:
            self.UpdateGroupDictList(name,skills)
        else:
            found_set = set()
            for s in skills:
                if s in self.skill_dict_list:
                    self.skill_dict_list[s].UpdateGroupSet({name})
                    found_set.add(s)  
                    #print(s,"added in",name,"group set.")
            self.group_dict_list[name] = Group(name,found_set)

    def UpdateGroupDictList(self,name,skills):
        if name in self.group_dict_list:
            found_set = set()
            for s in skills:
                if s in self.skill_dict_list:
                      found_set.add(s)  
            self.group_dict_list[name].UpdateSkillSet(found_set)
        else:
            self.AddGroupDictList(name,skills)

    def AddAliasList(self,key,value):
        if key not in self.alias_dict_list:
            self.alias_dict_list[key]=value
            #print(key,"alias as",value,"created.")
            
    def AddIgnoreList(self,name):
        self.ignore_set.add(name)

    def ExportSkillDictList(self):
        file_path = "skills.csv"
        with open(file_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Name", "Search Keyword","Resource Path","Groups","Prerequisites"])
            for key, value in self.skill_dict_list.items():
                name=key
                search = value.keyword_search
                path = value.resource_path
                groups =""
            
                for g in value.group_set:
                    groups += "["
                    groups += g
                    groups +="]"
                pre = ""
                for p in value.prerequisite:
                    pre += "["
                    pre += p
                    pre +="]"
                writer.writerow([name,search,path,groups,pre])
            file.close()

    def ExportGroupDictList(self):
        file_path = "groups.csv"
        with open(file_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Name", "Search Keyword","Resource Path","skills"])
            for key, value in self.group_dict_list.items():
                name=key
                search = value.keyword_search
                path = value.resource_path
                skills =""
                for s in value.skill_set:
                    skills += "["
                    skills += s
                    skills +="]"
                writer.writerow([name,search,path,skills])
            file.close()

    def LearningDocumentPrepare(self,company,skills):
        document = ""
        #new_list = [x for x in job_skills if x not in resume_skills]

        #check leetcode required

        # get all the learning resource reference link
        for s in skills:
            key = s
            if s in self.alias_dict_list:
                key = alias_dict_list.get(s)
                
            if key in self.skill_dict_list:
                # do something
                continue
            elif key in self.group_dict_list:
                # do something
                continue
            elif key in self.ignore_list:
                continue
            elif key in self.not_found_list:
                continue
            else:
                self.not_found_set.add(key)
        
        return document   




In [8]:
test = TechStack()
# top programming list
name_list = []
name_list.append("javascript")
name_list.append("html")
name_list.append("css")
name_list.append("python")
name_list.append("sql")
name_list.append("bash")
test.AddAliasList("shell","bash")
name_list.append("java")
name_list.append("c#")
name_list.append("c++")
name_list.append("c")
name_list.append("php")
name_list.append("powershell")
name_list.append("go")
name_list.append("rust")
name_list.append("kotlin")
name_list.append("ruby")
name_list.append("lua")
name_list.append("dart")
name_list.append("assembly")
name_list.append("swift")
name_list.append("r")
name_list.append("visual basic .net")
name_list.append("matlab")
name_list.append("vba")
test.AddAliasList("visual basic","vba")
name_list.append("groovy")
name_list.append("delphi")
name_list.append("scala")
name_list.append("perl")
name_list.append("elixir")
name_list.append("objective-c")
name_list.append("haskell")
name_list.append("gdscript")
name_list.append("lisp")
name_list.append("solidity")
name_list.append("clojure")
name_list.append("julia")
name_list.append("erlang")
name_list.append("f#")
name_list.append("fortran")
name_list.append("prolog")
name_list.append("zig")
name_list.append("ada")
name_list.append("ocaml")
name_list.append("sas")
name_list.append("crystal")
name_list.append("apl")
name_list.append("flow")
name_list.append("raku")
name_list.append("racket")
for i in range(len(name_list)):
    keyword = name_list[i] + " programming"
    test.AddSkillDictList(name_list[i], "code", keyword)
    

skill_set ={"c++","java","python","c","c#","javascript","typescript","php","swift","kotlin","dart","go","ruby","scala","rust","racket","erlang","elixir"}
test.AddGroupDictList("leetcode",skill_set)

# top framework/libraey
name_list.clear()
name_list.append(".net")
name_list.append("numpy")
name_list.append("pandas")
name_list.append(".net framework")
name_list.append("spring")
test.AddAliasList("spring framework","spring")
name_list.append("rabbitmq")
name_list.append("tensorflow")
name_list.append("scikit-learn")
test.AddAliasList("sklearn","scikit-learn")
name_list.append("flutter")
name_list.append("kafka")
test.AddAliasList("apache kafka","kafka")
name_list.append("torch")
test.AddAliasList("pytorch","torch")
name_list.append("react native")
name_list.append("opencv")
name_list.append("electron")
name_list.append("opengl")
name_list.append("qt")
name_list.append("cuda")
name_list.append("keras")
name_list.append("spark")
test.AddAliasList("apache spark","spark")
name_list.append("swiftui")
name_list.append("xamarin")
name_list.append("lonic")
name_list.append("transformers")
test.AddAliasList("hugging face","transformers")
name_list.append("gtk")
name_list.append("cordova")
name_list.append(".net maui")
name_list.append("hadoop")
name_list.append("tauri")
name_list.append("capacitor")
name_list.append("tidyverse")
name_list.append("quarkus")
name_list.append("ktor")
name_list.append("mfc")
name_list.append("jax")
name_list.append("micronaut")
name_list.append("uno platform")
for i in range(len(name_list)):
    keyword = name_list[i] + " framework/library"
    test.AddSkillDictList(name_list[i], "code", keyword)

#top frontend
name_list.clear()
name_list.append("react")
name_list.append("tailwind css")
name_list.append("angular")
name_list.append("vue.js")
name_list.append("bootstrap")
name_list.append("mdl")
test.AddAliasList("material design lite","mdl")
name_list.append("bulma")
name_list.append("mdl")
name_list.append("semantic ui")
name_list.append("foundation")
name_list.append("ulkit")
for i in range(len(name_list)):
    keyword = name_list[i] + " frontend"
    test.AddSkillDictList(name_list[i], "code", keyword, {"frontend"})

# top backend
name = "spring boot"
test.AddSkillDictList(name, "code", name + " backend", {"backend"},{"java"})
name = "django"
test.AddSkillDictList(name, "code", name + " backend", {"backend"},{"python"})
name = "express.js"
test.AddSkillDictList(name, "code", name + " backend", {"backend"},{"node.js"})
name = "laravel"
test.AddSkillDictList(name, "code", name + " backend", {"backend"},{"php"})
name = "ruby on rails"
test.AddSkillDictList(name, "code", name + " backend", {"backend"},{"ruby"})
name = "asp.net core"
test.AddSkillDictList(name, "code", name + " backend", {"backend"},{"c#"})
name = "nestjs"
test.AddSkillDictList(name, "code", name + " backend", {"backend"},{"typescript","node.js"})
name = "koa.js"
test.AddSkillDictList(name, "code", name + " backend", {"backend"},{"node.js"})
name = "phoenix"
test.AddSkillDictList(name, "code", name + " backend", {"backend"},{"elixir"})

# top web framework
name_list.clear()
name_list.append("node.js")
name_list.append("react")
name_list.append("jquery")
name_list.append("express")
name_list.append("angular")
name_list.append("next.js")
name_list.append("asp.net.core")
name_list.append("vue.js")
name_list.append("wordpress")
name_list.append("asp.net")
name_list.append("flask")
name_list.append("spring boot")
name_list.append("django")
name_list.append("laravel")
name_list.append("fastapi")
name_list.append("angularjs")
name_list.append("svelte")
name_list.append("ruby on rails")
name_list.append("nestjs")
name_list.append("blazor")
name_list.append("nuxt.js")
name_list.append("symfony")
name_list.append("deno")
name_list.append("gatsby")
name_list.append("fastify")
name_list.append("phoenix")
name_list.append("drupal")
name_list.append("codelgniter")
name_list.append("solid.js")
name_list.append("remix")
name_list.append("elm")
name_list.append("play")
test.AddAliasList("play framework","play")
name_list.append("lit")
name_list.append("qwik")
for i in range(len(name_list)):
    keyword = name_list[i] + " framework/library"
    test.AddSkillDictList(name_list[i], "code", keyword, {"web development"})

# top mobile development framework
name_list.clear()
name_list.append("flutter")
name_list.append("react native")
name_list.append("swiftic")
name_list.append("kotlin")
name_list.append("ionic")
name_list.append("xamarin")
name_list.append("corona")
test.AddAliasList("corona sdk","corona")
name_list.append("titanium")
test.AddAliasList("titanium sdk","titanium")
name_list.append("nativescript")
name_list.append("jquery mobile")
for i in range(len(name_list)):
    keyword = name_list[i] + " mobile development framework"
    test.AddSkillDictList(name_list[i], "code", keyword, {"mobile development"})

# top microservices
name = "spring boot"
test.AddSkillDictList(name, "code", name + " framework", {"microservices"},{"java"})
name = "spring cloud"
test.AddSkillDictList(name, "code", name + " framework", {"microservices"},{"java"})
name = "express.js"
test.AddSkillDictList(name, "code", name + " framework", {"microservices"},{"node.js"})
name = "django"
test.AddSkillDictList(name, "code", name + " framework", {"microservices"},{"python"})
name = "django rest"
test.AddSkillDictList(name, "code", name + " framework", {"microservices"},{"python"})
name = "go kit"
test.AddSkillDictList(name, "code", name + " framework", {"microservices"},{"go"})
test.AddAliasList("golang","go")
name = "micronaut"
test.AddSkillDictList(name, "code", name + " framework", {"microservices"},{"java","groovy","kotlin"})
name = "fastapi"
test.AddSkillDictList(name, "code", name + " framework", {"microservices"},{"python"})
name = "helidon"
test.AddSkillDictList(name, "code", name + " framework", {"microservices"},{"java"})
name = "lagom"
test.AddSkillDictList(name, "code", name + " framework", {"microservices"},{"java","scala"})
name = "quarkus"
test.AddSkillDictList(name, "code", name + " framework", {"microservices"},{"java"})

# top test automation frameworks
name = "selenium"
test.AddSkillDictList(name, "code", name + " framework", {"automation testing"},{"java","python","c#"})
name = "appium"
test.AddSkillDictList(name, "code", name + " framework", {"automation testing","mobile development"})
name = "testng"
test.AddSkillDictList(name, "code", name + " framework", {"automation testing"},{"java","junit"})
name = "junit"
test.AddSkillDictList(name, "code", name + " framework", {"automation testing"},{"java"})
name = "cucumber"
test.AddSkillDictList(name, "code", name + " framework", {"automation testing"})
name = "testcomplete"
test.AddSkillDictList(name, "code", name + " framework", {"automation testing"},{"javascript","python","visual basic"})
name = "robot"
test.AddSkillDictList(name, "code", name + " framework", {"automation testing"})
test.AddAliasList("robot framework","robot")
name = "cypress"
test.AddSkillDictList(name, "code", name + " framework", {"automation testing"},{"javascript"})
name = "junit jupiter"
test.AddSkillDictList(name, "code", name + " framework", {"automation testing"},{"java","junit"})
name = "pytest"
test.AddSkillDictList(name, "code", name + " framework", {"automation testing"},{"python"})
name = "rspec"
test.AddSkillDictList(name, "code", name + " framework", {"automation testing"})
name = "galen"
test.AddSkillDictList(name, "code", name + " framework", {"automation testing","web development"})

# top distributed
name = "hadoop"
test.AddSkillDictList(name, "code", name + " framework", {"distributed"})
name = "mapreduce"
test.AddSkillDictList(name, "code", name + " framework", {"distributed"})
name = "spark"
test.AddSkillDictList(name, "code", name + " framework", {"distributed"})
name = "hive"
test.AddSkillDictList(name, "code", name + " framework", {"distributed"})
test.AddAliasList("apache hive","hive")
name = "storm"
test.AddSkillDictList(name, "code", name + " framework", {"distributed"})
test.AddAliasList("apache storm","storm")
name = "samza"
test.AddSkillDictList(name, "code", name + " framework", {"distributed"})
test.AddAliasList("apache samza","samza")
name = "flink"
test.AddSkillDictList(name, "code", name + " framework", {"distributed"})
test.AddAliasList("apache flink","flink")
name = "heron"
test.AddSkillDictList(name, "code", name + " framework", {"distributed"})
test.AddAliasList("apache heron","heron")
name = "kudu"
test.AddSkillDictList(name, "code", name + " framework", {"distributed"})
test.AddAliasList("apache kudu","kudu")
name = "presto"
test.AddSkillDictList(name, "code", name + " framework", {"distributed"})
name = "zookeeper"
test.AddSkillDictList(name, "code", name + " framework", {"distributed"})
test.AddAliasList("apache zookeeper","zookeeper")

# mq
name = "rabbitmq"
test.AddSkillDictList(name, "code", name , {"mq"})
name = "ironmq"
test.AddSkillDictList(name, "code", name , {"mq"})
name = "ibm mq"
test.AddSkillDictList(name, "code", name , {"mq"})
name = "mulesoft"
test.AddSkillDictList(name, "code", name + " message queue" , {"mq"})
name = "pubsub+"
test.AddSkillDictList(name, "code", name + " message queue" , {"mq"})
name = "kafka"
test.AddSkillDictList(name, "code", name + " message queue" , {"mq"})
name = "zeromq"
test.AddSkillDictList(name, "code", name + " message queue" , {"mq"})
name = "tibco"
test.AddSkillDictList(name, "code", name + " message queue" , {"mq"})

# top ai framework
name = "torch"
test.AddSkillDictList(name, "code", name + " framework", {"ai"})
name = "scikit-learn"
test.AddSkillDictList(name, "code", name + " framework", {"ai"})
name = "tensorflow"
test.AddSkillDictList(name, "code", name + " framework", {"ai"})
name = "keras"
test.AddSkillDictList(name, "code", name + " framework", {"ai"})
name = "langchain"
test.AddSkillDictList(name, "code", name + " framework", {"ai"})
name = "transformers"
test.AddSkillDictList(name, "code", name + " framework", {"ai"})
name = "opennn"
test.AddSkillDictList(name, "code", name + " framework", {"ai"})
name = "openai"
test.AddSkillDictList(name, "code", name + " framework", {"ai"})
name = "pybrain"
test.AddSkillDictList(name, "code", name + " framework", {"ai"})
name = "ibm watson"
test.AddSkillDictList(name, "code", name + " framework", {"ai"})
name = "cntk"
test.AddSkillDictList(name, "code", name + " framework", {"ai"})
test.AddAliasList("microsoft cognitive toolkit","cntk")
name = "dl4j"
test.AddSkillDictList(name, "code", name + " framework", {"ai"})
test.AddAliasList("deeplearning4j","dl4j")
name = "theano"
test.AddSkillDictList(name, "code", name + " framework", {"ai"})
name = "mxnet"
test.AddSkillDictList(name, "code", name + " framework", {"ai"})
name = "caffe"
test.AddSkillDictList(name, "code", name + " framework", {"ai"})
name = "xgboost"
test.AddSkillDictList(name, "code", name + " framework", {"ai"})

# web crawler
name = "selenium"
test.AddSkillDictList(name, "code", name + " framework/library", {"crawler"})
name = "selenium base"
test.AddSkillDictList(name, "code", name + " framework/library", {"crawler"},{"selenium"})
name = "scrapy"
test.AddSkillDictList(name, "code", name + " framework/library", {"crawler"},{"python"})
name = "mechanicalsoup"
test.AddSkillDictList(name, "code", name + " framework/library", {"crawler"},{"python"})
name = "beautiful soup"
test.AddSkillDictList(name, "code", name + " framework/library", {"crawler"},{"python"})

#database
name = "oracle database"
test.AddSkillDictList(name, "database", name, {"database"})
name = "mysql"
test.AddSkillDictList(name, "database", name + " database", {"database","relational"})
name = "microsoft sql"
test.AddSkillDictList(name, "database", name + " database", {"database","warehouse"})
name = "postgresql"
test.AddSkillDictList(name, "database", name + " database", {"database","relational"})
name = "mongodb"
test.AddSkillDictList(name, "database", name + " database", {"database","nosql","document database"})
name = "cassandra"
test.AddSkillDictList(name, "database", name + " database", {"database","nosql","distributed"})
name = "redis"
test.AddSkillDictList(name, "database", name + " database", {"database","nosql","in memory database"})
name = "neo4j"
test.AddSkillDictList(name, "database", name + " database", {"database","nosql","graph database"})
name = "sqlite"
test.AddSkillDictList(name, "database", name + " database", {"database"})
name = "mariadb"
test.AddSkillDictList(name, "database", name + " database", {"database"})
name = "elasticsearch"
test.AddSkillDictList(name, "database", name + " database", {"database"})
name = "ibm db2"
test.AddSkillDictList(name, "database", name + " database", {"database"})
name = "couchbase"
test.AddSkillDictList(name, "database", name + " database", {"database"})
name = "orientdb"
test.AddSkillDictList(name, "database", name + " database", {"database","nosql"})
name = "firebirdsql"
test.AddSkillDictList(name, "database", name + " database", {"database","relational"})
name = "influxdb"
test.AddSkillDictList(name, "database", name + " database", {"database","time series"})
name = "kdb"
test.AddSkillDictList(name, "database", name + " database", {"database","time series"})
name = "prometheus"
test.AddSkillDictList(name, "database", name + " database", {"database","time series"})
name = "chroma"
test.AddSkillDictList(name, "database", name + " database", {"database","vector database"})
name = "pinecone"
test.AddSkillDictList(name, "database", name + " database", {"database","vector database"})
name = "faiss"
test.AddSkillDictList(name, "database", name + " database", {"database","vector database"})

#container
name = "docker"
test.AddSkillDictList(name, "container", name + " container",{"container"} )
name = "lxc"
test.AddSkillDictList(name, "container", name + " container",{"container"} )
name = "container linux"
test.AddSkillDictList(name, "container", name ,{"container"} )
name = "portainer"
test.AddSkillDictList(name, "container", name + " container",{"container"} )
name = "messos"
test.AddSkillDictList(name, "container", name + " container",{"container"} )

# cl/cd
name = "spacelift"
test.AddSkillDictList(name, "ci cd", name + " CI/CD",{"ci cd"} )
name = "github"
test.AddSkillDictList(name, "ci cd", name + " CI/CD",{"ci cd"} )
name = "jenkins"
test.AddSkillDictList(name, "ci cd", name + " CI/CD",{"ci cd"} )
name = "buddy"
test.AddSkillDictList(name, "ci cd", name + " CI/CD",{"ci cd"} )
name = "teamcity"
test.AddSkillDictList(name, "ci cd", name + " CI/CD",{"ci cd"} )
name = "circleci"
test.AddSkillDictList(name, "ci cd", name + " CI/CD",{"ci cd"} )
name = "tracisci"
test.AddSkillDictList(name, "ci cd", name + " CI/CD",{"ci cd"} )
name = "gitlab"
test.AddSkillDictList(name, "ci cd", name + " CI/CD",{"ci cd"} )
name = "bitbucket"
test.AddSkillDictList(name, "ci cd", name + " CI/CD",{"ci cd"} )
name = "harness"
test.AddSkillDictList(name, "ci cd", name + " CI/CD",{"ci cd"} )
name = "semaphore"
test.AddSkillDictList(name, "ci cd", name + " CI/CD",{"ci cd"} )

# monitoring tool
name = "datadog"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )
name = "grafana"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )
name = "nagios"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )
name = "micro focus sitescope"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )
name = "sensu"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )
name = "prometheus"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )
name = "observium"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )
name = "checkmk"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )
name = "new relic"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )
name = "dynatrace"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )
name = "atera"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )
name = "paessler prtg"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )
name = "zabbix"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )
name = "appdynamics"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )
name = "logrocket"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )
name = "logz.io"
test.AddSkillDictList(name, "monitoring", name,{"monitoring"} )

#aws
name = "aws"
test.AddSkillDictList(name, "aws", name + " cloud", {"cloud","aws"})
name = "s3"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud","aws","storage"})
name = "efs"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud","aws"})
name = "fsx"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud","aws"})
name = "ebs"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud","aws"})
name = "amazon file cache"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud","aws"})
name = "aws drs"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud","aws"})
name = "neptune"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud","aws","database","graph database"})
name = "rds"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud","aws","database","relational"})
name = "dynamodb"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud","aws","database","nosql"})
name = "redshift"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud","aws","warehouse"})
name = "aws database migration service"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud","aws"})
name = "elasticache"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud","aws","data store"})
name = "memorydb"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud","aws","database","in memory database"})
name = "documentdb"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud","aws","database","document databaset"})
name = "timestream"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud","aws","database","time series"})
name = "aws glue"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud","aws"})
name = "athena"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud","aws","big data"})
name = "emr"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud","aws","big data"})
name = "amazon sns"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud","aws","mq","big data"})
name = "lake formation"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud","aws"})
name = "sagemaker"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud","aws","ai"})
name = "ecs"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud","aws","container"})
name = "fargate"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud","aws","container"})
name = "lambda"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud","aws"})
name = "cloudwatch"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud","aws","monitoring"})
name = "sqs"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud","aws","mq"})
name = "codepipeline"
test.AddSkillDictList(name, "aws", "aws " + name, {"cloud","aws","ci cd"})

#azure
name = "azure"
test.AddSkillDictList(name, "azure", name + " cloud", {"cloud","azure"})
name = "cosmosdb"
test.AddSkillDictList(name, "azure", "azure " + name, {"cloud","azure","database","nosql"})
name = "azure sql database"
test.AddSkillDictList(name, "azure",  name, {"cloud","azure","database","relational"})
name = "azure blobs"
test.AddSkillDictList(name, "azure", name, {"cloud","azure","storage"})
name = "azure file"
test.AddSkillDictList(name, "azure", name, {"cloud","azure"})
name = "azure elastic san"
test.AddSkillDictList(name, "azure",  name, {"cloud","azure"})
name = "azure tables"
test.AddSkillDictList(name, "azure",  name, {"cloud","azure"})
name = "azure queues"
test.AddSkillDictList(name, "azure", name, {"cloud","azure"})
name = "azure managed disks"
test.AddSkillDictList(name, "azure",  name, {"cloud","azure"})
name = "azure container storage"
test.AddSkillDictList(name, "azure",  name, {"cloud","azure"})
name = "azure container"
test.AddSkillDictList(name, "azure", name, {"cloud","azure","container"})
name = "azure data lake"
test.AddSkillDictList(name, "azure",  name, {"cloud","azure"})
name = "service bus"
test.AddSkillDictList(name, "azure", "azure " + name, {"cloud","azure","mq"})
name = "hdinsight"
test.AddSkillDictList(name, "azure", "azure " + name, {"cloud","azure","mq"})
name = "azure synapse analytics"
test.AddSkillDictList(name, "azure",  name, {"cloud","azure","warehouse","big data"})
name = "azure ai"
test.AddSkillDictList(name, "azure",  name, {"cloud","azure","ai"})
name = "azure montior"
test.AddSkillDictList(name, "azure",  name, {"cloud","azure","monitoring"})
name = "azure pipelines"
test.AddSkillDictList(name, "azure",  name, {"cloud","azure","ci cd"})

#google cloud
name = "google cloud"
test.AddSkillDictList(name, "google cloud", name, {"cloud","google cloud"})
name = "cloud sql"
test.AddSkillDictList(name, "google cloud", "google " + name, {"cloud","google cloud","database","relational"})
name = "alloydb"
test.AddSkillDictList(name, "google cloud", "google " + name, {"cloud","google cloud","database","relational"})
name = "spanner"
test.AddSkillDictList(name, "google cloud", "google " + name, {"cloud","google cloud","database","relational"})
name = "bigtable"
test.AddSkillDictList(name, "google cloud", "google " + name, {"cloud","google cloud","database","key value"})
name = "firestore"
test.AddSkillDictList(name, "google cloud", "google " + name, {"cloud","google cloud","database","document database"})
name = "firebase realtime database"
test.AddSkillDictList(name, "google cloud", "google " + name, {"cloud","google cloud","database","document database"})
name = "memorystore"
test.AddSkillDictList(name, "google cloud", "google " + name, {"cloud","google cloud","database","in memory database"})
name = "mongodb atlas"
test.AddSkillDictList(name, "google cloud", "google " + name, {"cloud","google cloud","database","nosql"})
name = "bigquery"
test.AddSkillDictList(name, "google cloud", "google " + name, {"cloud","google cloud","warehouse","big data"})
name = "dataproc"
test.AddSkillDictList(name, "google cloud", "google " + name, {"cloud","google cloud"})
name = "google cloud storage"
test.AddSkillDictList(name, "google cloud", name, {"cloud","google cloud","storage"})
name = "vertex ai"
test.AddSkillDictList(name, "google cloud", name, {"cloud","google cloud","ai"})
name = "cloud tasks"
test.AddSkillDictList(name, "google cloud", "google " + name, {"cloud","google cloud","mq"})
name = "pub sub"
test.AddSkillDictList(name, "google cloud", "google " + name, {"cloud","google cloud","mq"})
name = "gke"
test.AddSkillDictList(name, "google cloud", "google " + name, {"cloud","google cloud","container"})
name = "google cloud monitoring"
test.AddSkillDictList(name, "google cloud", name, {"cloud","google cloud","monitoring"})
name = "cloud buid"
test.AddSkillDictList(name, "google cloud", "google "+ name, {"cloud","google cloud","ci cd"})

name = "snowflake"
test.AddSkillDictList(name, "other", name )
name = "databricks"
test.AddSkillDictList(name, "other", name )
name = "terraform"
test.AddSkillDictList(name, "other", name )
name = "postman"
test.AddSkillDictList(name, "other", name,{"monitoring","automation testing","web development"} )

# pen testing
name = "nmap"
test.AddSkillDictList(name, "penetration testing", name + " pen test", {"penetration testing"})
name = "w3af"
test.AddSkillDictList(name, "penetration testing", name +" pen test" , {"penetration testing"})
name = "tenable nessus"
test.AddSkillDictList(name, "penetration testing", name + " pen test", {"penetration testing"})
name = "onsecurity"
test.AddSkillDictList(name, "penetration testing", name + " pen test", {"penetration testing"})
name = "wireshark"
test.AddSkillDictList(name, "penetration testing", name + " pen test", {"penetration testing"})
name = "zap"
test.AddSkillDictList(name, "penetration testing", name + " pen test", {"penetration testing"})
name = "nikto"
test.AddSkillDictList(name, "penetration testing", name + " pen test", {"penetration testing"})
name = "cobalt"
test.AddSkillDictList(name, "penetration testing", name + " pen test", {"penetration testing"})
name = "rapid7"
test.AddSkillDictList(name, "penetration testing", name + " pen test", {"penetration testing"})
name = "astra security"
test.AddSkillDictList(name, "penetration testing", name + " pen test", {"penetration testing"})

# top blockchain
name = "ethereum"
test.AddSkillDictList(name, "blockchain", name + "blockchain", {"blockchain"})
name = "ethereum 2.0"
test.AddSkillDictList(name, "blockchain", name + "blockchain", {"blockchain"})
name = "ibm blockchain"
test.AddSkillDictList(name, "blockchain", name + "blockchain", {"blockchain"})
name = "hyperledger"
test.AddSkillDictList(name, "blockchain", name + "blockchain", {"blockchain"})
name = "eos"
test.AddSkillDictList(name, "blockchain", name + "blockchain", {"blockchain"})
name = "corda"
test.AddSkillDictList(name, "blockchain", name + "blockchain", {"blockchain"})
name = "ripple"
test.AddSkillDictList(name, "blockchain", name + "blockchain", {"blockchain"})
name = "quorum"
test.AddSkillDictList(name, "blockchain", name + "blockchain", {"blockchain"})
name = "iota"
test.AddSkillDictList(name, "blockchain", name + "blockchain", {"blockchain"})
name = "neo"
test.AddSkillDictList(name, "blockchain", name + "blockchain", {"blockchain"})
name = "tezos"
test.AddSkillDictList(name, "blockchain", name + "blockchain", {"blockchain"})
name = "stellar"
test.AddSkillDictList(name, "blockchain", name + "blockchain", {"blockchain"})
name = "sui network"
test.AddSkillDictList(name, "blockchain", name + "blockchain", {"blockchain"})
name = "hedera"
test.AddSkillDictList(name, "blockchain", name + "blockchain", {"blockchain"})
name = "aptos"
test.AddSkillDictList(name, "blockchain", name + "blockchain", {"blockchain"})
name = "flow"
test.AddSkillDictList(name, "blockchain", name + "blockchain", {"blockchain"})
name = "fantom"
test.AddSkillDictList(name, "blockchain", name + "blockchain", {"blockchain"})
name = "solana"
test.AddSkillDictList(name, "blockchain", name + "blockchain", {"blockchain"})
name = "tron"
test.AddSkillDictList(name, "blockchain", name + "blockchain", {"blockchain"})
name = "binance smart chain"
test.AddSkillDictList(name, "blockchain", name + "blockchain", {"blockchain"})
name = "icp"
test.AddSkillDictList(name, "blockchain", name + "blockchain", {"blockchain"})

# top game developer
name = "unreal engine"
test.AddSkillDictList(name, "game", name + "engine", {"game development"},{"c++"})
name = "unity"
test.AddSkillDictList(name, "game", name + "engine", {"game development"},{"c#"})
name = "godot"
test.AddSkillDictList(name, "game", name + "engine", {"game development"},{"c#"})
name = "cocos2d"
test.AddSkillDictList(name, "game", name + "engine", {"game development"})
name = "rpg maker"
test.AddSkillDictList(name, "game", name + "engine", {"game development"})
name = "vulkan"
test.AddSkillDictList(name, "game", name + "graphics framework", {"game development"})
name = "opengl"
test.AddSkillDictList(name, "game", name + "graphics framework", {"game development"})
name = "directx"
test.AddSkillDictList(name, "game", name + "graphics framework", {"game development"})
name = "physx"
test.AddSkillDictList(name, "game", name + "physics engine", {"game development"})
name = "bullet"
test.AddSkillDictList(name, "game", name + "physics engine", {"game development"})
name = "open dynamics engine"
test.AddSkillDictList(name, "game", name + "physics engine", {"game development"})
name = "fmod"
test.AddSkillDictList(name, "game", name + "audio framework", {"game development"})
name = "glm"
test.AddSkillDictList(name, "game", name + "math framework", {"game development"})
name = "assimp"
test.AddSkillDictList(name, "game", name + "library", {"game development"})

#quantum
name = "tensorflow quantum"
test.AddSkillDictList(name, "quantum", name , {"quantum"})
name = "microsoft quantum"
test.AddSkillDictList(name, "quantum", name , {"quantum"})
name = "qiskit"
test.AddSkillDictList(name, "quantum", name , {"quantum"})
name = "cuquantum"
test.AddSkillDictList(name, "quantum", name , {"quantum"})
test.ExportSkillDictList()
test.ExportGroupDictList()